In [20]:
from schema import *

s = Schema()

# Load up the CSVs
cardnum = pd.read_csv("./csv/bonuses/cardnum.csv").set_index("val_id")
person = pd.read_csv("./csv/bonuses/person.csv").set_index("cardnum")

# Insert them into the data frames
s.insert_dataframe(cardnum, "cardnum")
s.insert_dataframe(person, "person")

# Get handles on the nodes now in the schema - may be slightly ugly but
# we can slap on a load of syntactic sugar
c_cardnum = SchemaNode("cardnum", cluster="cardnum")
c_val_id = SchemaNode("val_id", cluster="cardnum")

p_cardnum = SchemaNode("cardnum", cluster="person")
p_person = SchemaNode("person", cluster="person")

s.blend(c_cardnum, p_cardnum, under="Cardnum")

In [21]:
cardnum

,cardnum
val_id,
1,5172
2,2354
3,1410
4,1111
5,2354
8,4412


In [22]:
person

,person
cardnum,
1111,Steve
1410,Tom
2354,Steve
6440,Harry
5467,Dick


In [23]:
from tables import *
table1 = s.get(['cardnum.val_id'])
print(table1)

[cardnum.val_id || ]
Empty DataFrame
Columns: []
Index: [1, 2, 3, 4, 5, 8]


In [24]:
table2 = table1.infer(['cardnum.val_id'], 'cardnum.cardnum')
print(table2)

[cardnum.val_id || cardnum.cardnum]
                cardnum.cardnum
cardnum.val_id                 
1                          5172
2                          2354
3                          1410
4                          1111
5                          2354
8                          4412


In [25]:
table3 = table2.infer(['cardnum.cardnum'], 'person.cardnum')
print(table3)

[cardnum.val_id || cardnum.cardnum person.cardnum]
                cardnum.cardnum  person.cardnum
cardnum.val_id                                 
1                          5172            5172
2                          2354            2354
5                          2354            2354
3                          1410            1410
4                          1111            1111
8                          4412            4412


In [26]:
table4 = table3.infer(['person.cardnum'], 'person.person')
print(table4)

[cardnum.val_id || cardnum.cardnum person.cardnum person.person]
                cardnum.cardnum  person.cardnum person.person
cardnum.val_id                                               
1.0                      5172.0            5172           NaN
2.0                      2354.0            2354         Steve
5.0                      2354.0            2354         Steve
3.0                      1410.0            1410           Tom
4.0                      1111.0            1111         Steve
8.0                      4412.0            4412           NaN
4 values hidden


In [27]:
table = table4.hide('person.cardnum').hide('cardnum.cardnum')
print(table)

[cardnum.val_id || person.person]
               person.person
cardnum.val_id              
2.0                    Steve
5.0                    Steve
3.0                      Tom
4.0                    Steve
2 keys hidden
2 values hidden


In [28]:
table1 = s.get(['cardnum.val_id'])
print(table1)

[cardnum.val_id || ]
Empty DataFrame
Columns: []
Index: [1, 2, 3, 4, 5, 8]


In [29]:
table2 = table1.infer(['cardnum.val_id'], 'person.person')
print(table2)

[cardnum.val_id || person.person]
               person.person
cardnum.val_id              
2.0                    Steve
3.0                      Tom
4.0                    Steve
5.0                    Steve
2 keys hidden
2 values hidden


In [30]:
table3 = table2.set_key(["person.person"])
print(table3)

[person.person || cardnum.val_id]
                cardnum.val_id
person.person                 
Steve          [2.0, 4.0, 5.0]
Tom                      [3.0]
3 keys hidden


In [31]:
table4 = table3.show("cardnum.val_id")
print(table4)

[person.person cardnum.val_id || cardnum.val_id]
                              cardnum.val_id
person.person cardnum.val_id                
Steve         2.0                        2.0
Tom           3.0                        3.0
Steve         4.0                        4.0
              5.0                        5.0
2 keys hidden
2 values hidden


In [32]:
table5 = table4.hide("cardnum.val_id")
print(table5)

[person.person || cardnum.val_id]
                cardnum.val_id
person.person                 
Steve          [2.0, 4.0, 5.0]
Tom                      [3.0]
3 keys hidden


In [33]:
table6 = table5.set_key(["cardnum.val_id"])
print(table6)

[cardnum.val_id || person.person]
               person.person
cardnum.val_id              
2.0                    Steve
3.0                      Tom
4.0                    Steve
5.0                    Steve
2 keys hidden
2 values hidden


In [34]:
table = (s.get(['person.cardnum']).infer(['person.cardnum'], 'person.person'))
print(table)

[person.cardnum || person.person]
               person.person
person.cardnum              
1111                   Steve
1410                     Tom
2354                   Steve
6440                   Harry
5467                    Dick


In [35]:
tbl1 = table.compose(['cardnum.val_id'], 'person.cardnum')
print(table1)

table2 = tbl1.set_key(["person.person"])


[cardnum.val_id || ]
Empty DataFrame
Columns: []
Index: [1, 2, 3, 4, 5, 8]


In [36]:
table1 = table

In [37]:
table = s.get(['person.person']).infer(['person.person'], 'person.cardnum')
print(table)

person.cardnum
[person.person || person.cardnum]
              person.cardnum
person.person               
Steve           [1111, 2354]
Tom                   [1410]
Harry                 [6440]
Dick                  [5467]


In [38]:
table1 = table.show('person.cardnum')
print(table1)
table2 = table1.hide('person.cardnum')
print(table2)

[person.person person.cardnum || person.cardnum]
                              person.cardnum
person.person person.cardnum                
Steve         1111                      1111
              2354                      2354
Tom           1410                      1410
Harry         6440                      6440
Dick          5467                      5467


[person.person || person.cardnum]
              person.cardnum
person.person               
Steve           [1111, 2354]
Tom                   [1410]
Harry                 [6440]
Dick                  [5467]


In [39]:
s = Schema()
cardnum = pd.read_csv("./csv/bonuses/cardnum.csv").set_index("val_id")
tstart = pd.read_csv("./csv/bonuses/tstart.csv").set_index("val_id")
s.insert_dataframe(cardnum, "cardnum")
s.insert_dataframe(tstart, "tstart")

c_cardnum = SchemaNode("cardnum", cluster="cardnum")
c_val_id = SchemaNode("val_id", cluster="cardnum")
t_val_id = SchemaNode("val_id", cluster="tstart")
t_tstart = SchemaNode("tstart", cluster="tstart")

s.blend(c_val_id, t_val_id, under="Val_id")

In [40]:
t1 = s.get(["tstart.val_id"]).infer(["tstart.val_id"], "tstart.tstart")
print(t1)

[tstart.val_id || tstart.tstart]
                     tstart.tstart
tstart.val_id                     
1              2023-01-01 09:50:00
2              2023-01-01 11:10:00
3              2023-01-01 15:32:00
4              2023-01-01 15:34:00
5              2023-01-01 20:11:00
6              2023-01-01 21:17:00
7              2023-01-02 05:34:00


In [41]:
t1 = s.get(["Val_id"]).infer(["Val_id"], "tstart.tstart")
print(t1)

[Val_id || tstart.tstart]
              tstart.tstart
Val_id                     
1       2023-01-01 09:50:00
2       2023-01-01 11:10:00
3       2023-01-01 15:32:00
4       2023-01-01 15:34:00
5       2023-01-01 20:11:00
6       2023-01-01 21:17:00
7       2023-01-02 05:34:00
1 keys hidden


In [42]:
s = Schema()

cardnum = pd.read_csv("./csv/bonuses/cardnum.csv").set_index("val_id")
tstart = pd.read_csv("./csv/bonuses/tstart.csv").set_index("val_id")
bonus = pd.read_csv("./csv/bonuses/bonus.csv").set_index(["val_id", "cardnum"])

s.insert_dataframe(cardnum, "cardnum")
s.insert_dataframe(tstart, "tstart")
s.insert_dataframe(bonus, "bonus")

c_cardnum = SchemaNode("cardnum", cluster="cardnum")
b_cardnum = SchemaNode("cardnum", cluster="bonus")
c_val_id = SchemaNode("val_id", cluster="cardnum")
t_val_id = SchemaNode("val_id", cluster="tstart")
b_val_id = SchemaNode("val_id", cluster="bonus")

s.blend(c_val_id, t_val_id, under ="Val_id")
s.blend(c_val_id, b_val_id)
s.blend(c_cardnum, b_cardnum, under ="Cardnum")

In [43]:
t1 = s.get(["cardnum.val_id"]).infer(["cardnum.val_id"], "cardnum.cardnum")
print(t1)


[cardnum.val_id || cardnum.cardnum]
                cardnum.cardnum
cardnum.val_id                 
1                          5172
2                          2354
3                          1410
4                          1111
5                          2354
8                          4412


In [44]:
t2 = t1.infer(["cardnum.val_id"], "bonus.bonus")
print(t2)

[cardnum.val_id || cardnum.cardnum bonus.bonus]
                cardnum.cardnum  bonus.bonus
cardnum.val_id                              
1                          5172  [4.0, 12.0]
2                          2354   [5.0, 7.0]
3                          1410        [1.0]
4                          1111           []
5                          2354        [2.0]
8                          4412           []


In [45]:
t3 = t2.set_key(["cardnum.val_id", "cardnum.cardnum"])
print(t3)

[cardnum.val_id cardnum.cardnum || bonus.bonus]
                                bonus.bonus
cardnum.val_id cardnum.cardnum             
1              5172             [4.0, 12.0]
2              2354              [5.0, 7.0]
3              1410                   [1.0]
5              2354                   [2.0]
2 keys hidden


In [46]:
t4 = t3.show("bonus.cardnum")
print(t4)

[cardnum.val_id cardnum.cardnum bonus.cardnum || bonus.bonus]
                                              bonus.bonus
cardnum.val_id cardnum.cardnum bonus.cardnum             
1              5172            5172.0                 4.0
                               1410.0                12.0
2              2354            1111.0                 5.0
                               6440.0                 7.0
3              1410            1111.0                 1.0
5              2354            1410.0                 2.0
2 keys hidden


In [47]:
t6 = t4.filter((t4["cardnum.cardnum"] == 2354) | (t4["bonus.cardnum"] == 1111))
t7 = t6.assign("bonuspo", t6["bonus.bonus"] + 1)


In [48]:
t8 = t6.assign("bonustimestwo", t6["bonus.bonus"] + t6["bonus.bonus"])
print(t8)

[cardnum.val_id cardnum.cardnum bonus.cardnum || bonus.bonus bonustimestwo]
                                              bonus.bonus  bonustimestwo
cardnum.val_id cardnum.cardnum bonus.cardnum                            
2              2354            1111.0                 5.0           10.0
                               6440.0                 7.0           14.0
3              1410            1111.0                 1.0            2.0
5              2354            1410.0                 2.0            4.0


In [49]:
t8 = t6.assign("bonusplusvalid", t6["bonus.bonus"] + t6["cardnum.val_id"])
print(t8)

[cardnum.val_id cardnum.cardnum bonus.cardnum || bonus.bonus bonusplusvalid]
                                              bonus.bonus  bonusplusvalid
cardnum.val_id cardnum.cardnum bonus.cardnum                             
2              2354            1111.0                 5.0             7.0
                               6440.0                 7.0             9.0
3              1410            1111.0                 1.0             4.0
5              2354            1410.0                 2.0             7.0


In [50]:
t5 = t4.equate("cardnum.cardnum", "bonus.cardnum")
print(t5)

[cardnum.val_id cardnum.cardnum || bonus.bonus]
                                bonus.bonus
cardnum.val_id cardnum.cardnum             
1              5172                     4.0


In [5]:
from schema import *
import pandas as pd

flows = pd.read_csv("./csv/flows/flows.csv").set_index(["from_city", "to_city"])

flows

s = Schema()
s.insert_dataframe(flows, "flows")
s.blend(SchemaNode("from_city", cluster="flows"), SchemaNode("to_city", cluster="flows"), under="City")

In [7]:
flows

volume
from_city to_city          
Cambridge London        3.0
          Edinburgh     2.4
          Oxford        0.6
London    Cambridge     4.2
          Oxford        4.8
Oxford    London        1.8
          Edinburgh     1.2
Edinburgh Cambridge     0.6
          London        2.4

In [6]:
t1 = s.get(["City", "City"], ["FromCity", "ToCity"])
print(t1)

[FromCity ToCity || ]
Empty DataFrame
Columns: []
Index: [(Cambridge, Cambridge), (Cambridge, London), (Cambridge, Oxford), (Cambridge, Edinburgh), (London, Cambridge), (London, London), (London, Oxford), (London, Edinburgh), (Oxford, Cambridge), (Oxford, London), (Oxford, Oxford), (Oxford, Edinburgh), (Edinburgh, Cambridge), (Edinburgh, London), (Edinburgh, Oxford), (Edinburgh, Edinburgh)]


In [8]:
t2 = t1.infer(["FromCity", "ToCity"], "flows.volume")
print(t2)

[FromCity ToCity || flows.volume]
                     flows.volume
FromCity  ToCity                 
Cambridge London              3.0
          Edinburgh           2.4
          Oxford              0.6
London    Cambridge           4.2
          Oxford              4.8
Oxford    London              1.8
          Edinburgh           1.2
Edinburgh Cambridge           0.6
          London              2.4
7 keys hidden


In [10]:
fill_na = lambda x: 0 if pd.isnull(x) else x
t3 = t2.assign("volume", t2["flows.volume"].apply(fill_na, Cardinality.ONE_TO_ONE)).sort(
             ["FromCity", "ToCity"]).hide("flows.volume")
print(t3)

[FromCity ToCity || volume]
                     volume
FromCity  ToCity           
Cambridge Cambridge     0.0
          Edinburgh     2.4
          London        3.0
          Oxford        0.6
Edinburgh Cambridge     0.6
          Edinburgh     0.0
          London        2.4
          Oxford        0.0
London    Cambridge     4.2
          Edinburgh     0.0
          London        0.0
          Oxford        4.8
Oxford    Cambridge     0.0
          Edinburgh     1.2
          London        1.8
          Oxford        0.0


In [11]:
t4 = t3.set_key(["ToCity"])
print(t4)

[ToCity || FromCity volume]
                                         FromCity                volume
ToCity                                                                 
Cambridge  [Cambridge, Edinburgh, London, Oxford]  [0.0, 0.6, 4.2, 0.0]
Edinburgh  [Cambridge, Edinburgh, London, Oxford]  [2.4, 0.0, 0.0, 1.2]
London     [Cambridge, Edinburgh, London, Oxford]  [3.0, 2.4, 0.0, 1.8]
Oxford     [Cambridge, Edinburgh, London, Oxford]  [0.6, 0.0, 4.8, 0.0]


In [12]:
t5 = t4.assign("total_inflow", t4["volume"].aggregate(sum)).sort(["FromCity", "ToCity"])
print(t5)

[ToCity || FromCity volume total_inflow]
                                         FromCity                volume  \
ToCity                                                                    
Cambridge  [Cambridge, Edinburgh, London, Oxford]  [0.0, 0.6, 4.2, 0.0]   
Edinburgh  [Cambridge, Edinburgh, London, Oxford]  [2.4, 0.0, 0.0, 1.2]   
London     [Cambridge, Edinburgh, London, Oxford]  [3.0, 2.4, 0.0, 1.8]   
Oxford     [Cambridge, Edinburgh, London, Oxford]  [0.6, 0.0, 4.8, 0.0]   

           total_inflow  
ToCity                   
Cambridge           4.8  
Edinburgh           3.6  
London              7.2  
Oxford              5.4  


In [13]:
t6 = t3.set_key(["FromCity"])
print(t6)
t7 = t6.assign("total_outflow", t6["volume"].aggregate(sum)).sort(["FromCity", "ToCity"])
print(t7)

[FromCity || ToCity volume]
                                           ToCity                volume
FromCity                                                               
Cambridge  [Cambridge, Edinburgh, London, Oxford]  [0.0, 2.4, 3.0, 0.6]
Edinburgh  [Cambridge, Edinburgh, London, Oxford]  [0.6, 0.0, 2.4, 0.0]
London     [Cambridge, Edinburgh, London, Oxford]  [4.2, 0.0, 0.0, 4.8]
Oxford     [Cambridge, Edinburgh, London, Oxford]  [0.0, 1.2, 1.8, 0.0]


[FromCity || ToCity volume total_outflow]
                                           ToCity                volume  \
FromCity                                                                  
Cambridge  [Cambridge, Edinburgh, London, Oxford]  [0.0, 2.4, 3.0, 0.6]   
Edinburgh  [Cambridge, Edinburgh, London, Oxford]  [0.6, 0.0, 2.4, 0.0]   
London     [Cambridge, Edinburgh, London, Oxford]  [4.2, 0.0, 0.0, 4.8]   
Oxford     [Cambridge, Edinburgh, London, Oxford]  [0.0, 1.2, 1.8, 0.0]   

           total_outflow  
FromCity          

In [14]:
 t8 = t3.infer(["ToCity"], "total_inflow")
print(t8)
t9 = t8.infer(["FromCity"], "total_outflow")
# TODO: Define aggregation over columns with no hidden keys
print(t9)

[FromCity ToCity || volume total_inflow]
                     volume  total_inflow
FromCity  ToCity                         
Cambridge Cambridge     0.0           4.8
Edinburgh Cambridge     0.6           4.8
London    Cambridge     4.2           4.8
Oxford    Cambridge     0.0           4.8
Cambridge Edinburgh     2.4           3.6
Edinburgh Edinburgh     0.0           3.6
London    Edinburgh     0.0           3.6
Oxford    Edinburgh     1.2           3.6
Cambridge London        3.0           7.2
Edinburgh London        2.4           7.2
London    London        0.0           7.2
Oxford    London        1.8           7.2
Cambridge Oxford        0.6           5.4
Edinburgh Oxford        0.0           5.4
London    Oxford        4.8           5.4
Oxford    Oxford        0.0           5.4


[FromCity ToCity || volume total_inflow total_outflow]
                     volume  total_inflow  total_outflow
FromCity  ToCity                                        
Cambridge Cambridge     0.0     

In [15]:
t10 = (t9.assign("relative_outflow", t9["volume"] / t9["total_outflow"])
                 .assign("expected_outflow", t9["total_inflow"] / 21))
print(t10)

[FromCity ToCity || volume total_inflow total_outflow relative_outflow expected_outflow]
                     volume  total_inflow  total_outflow  relative_outflow  \
FromCity  ToCity                                                             
Cambridge Cambridge     0.0           4.8            6.0          0.000000   
          Edinburgh     2.4           3.6            6.0          0.400000   
          London        3.0           7.2            6.0          0.500000   
          Oxford        0.6           5.4            6.0          0.100000   
Edinburgh Cambridge     0.6           4.8            3.0          0.200000   
          Edinburgh     0.0           3.6            3.0          0.000000   
          London        2.4           7.2            3.0          0.800000   
          Oxford        0.0           5.4            3.0          0.000000   
London    Cambridge     4.2           4.8            9.0          0.466667   
          Edinburgh     0.0           3.6            

In [16]:
t11 = t10.set_key(["FromCity"]).hide("volume").hide("total_outflow").hide("ToCity").hide("total_inflow")
print(t11)

[FromCity || relative_outflow expected_outflow]
                                            relative_outflow  \
FromCity                                                       
Cambridge  [0.0, 0.39999999999999997, 0.5, 0.099999999999...   
Edinburgh  [0.19999999999999998, 0.0, 0.7999999999999999,...   
London     [0.4666666666666667, 0.0, 0.0, 0.5333333333333...   
Oxford                  [0.0, 0.39999999999999997, 0.6, 0.0]   

                                            expected_outflow  
FromCity                                                      
Cambridge  [0.22857142857142856, 0.1714285714285714, 0.34...  
Edinburgh  [0.22857142857142856, 0.1714285714285714, 0.34...  
London     [0.22857142857142856, 0.1714285714285714, 0.34...  
Oxford     [0.22857142857142856, 0.1714285714285714, 0.34...  

